<a href="https://colab.research.google.com/github/PARKDAESAN/3rd-umc-pnu-front-study/blob/main/%EA%B3%BC%EC%A0%9C5_201927517_%EB%B0%95%EB%8C%80%EC%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [257]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import scipy.stats as st
from google.colab import drive

In [258]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [278]:
data_path = "/content/drive/My Drive/산업데이터과학/data/car.csv" 
data = pd.read_csv(data_path)
data.head()

,Year,Kilometers_Driven,Fuel_Type,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Maker,Price
0,2015,41000,Diesel,First,19.67,1582,126.20,5.0,NaN,Hyundai,12.50
1,2011,46000,Petrol,First,18.20,1199,88.70,5.0,8.61,Honda,4.50
2,2012,87000,Diesel,First,20.77,1248,88.76,7.0,NaN,Maruti,6.00
3,2013,40670,Diesel,Second,15.20,1968,140.80,5.0,NaN,Audi,17.74
4,2013,86999,Diesel,First,23.08,1461,63.10,5.0,NaN,Nissan,3.50


# [과제] 회귀 모형을 이용하여 과제용 데이터(실제값이 제거된 데이터) 의 예측치 구하기

> 일부 변수는 본인 의사에 맞게 수정해되 됨

> 변수 선택은 주관적으로 시행해도됨 (불필요하다고 생각되면 제거해도됨)

In [279]:
col = data.columns
print("전체 데이터 수는 {}개 입니다.".format(len(data)))
for col_value in col:
  detect_null = sum(data[col_value].isnull())
  print("{}에 포함된 결측값은 {}개 입니다".format(col_value, detect_null))

전체 데이터 수는 5558개 입니다.
Year에 포함된 결측값은 0개 입니다
Kilometers_Driven에 포함된 결측값은 0개 입니다
Fuel_Type에 포함된 결측값은 0개 입니다
Owner_Type에 포함된 결측값은 0개 입니다
Mileage에 포함된 결측값은 0개 입니다
Engine에 포함된 결측값은 0개 입니다
Power에 포함된 결측값은 0개 입니다
Seats에 포함된 결측값은 2개 입니다
New_Price에 포함된 결측값은 4785개 입니다
Maker에 포함된 결측값은 0개 입니다
Price에 포함된 결측값은 0개 입니다


In [281]:
#결측값이 많은 New_Price 제거
data_not_nullcolumns=['Year', 'Kilometers_Driven', 'Fuel_Type', 'Owner_Type', 'Mileage', 'Engine', 'Power', 'Seats', 'Maker', 'Price']
data=data.loc[:,data_not_nullcolumns]
data['Year']=data['Year'].astype('object')
data['Seats']=data["Seats"].astype('object')
print(data.dtypes)
data = pd.get_dummies(data,prefix_sep='_',drop_first=True)

Year                  object
Kilometers_Driven      int64
Fuel_Type             object
Owner_Type            object
Mileage              float64
Engine                 int64
Power                float64
Seats                 object
Maker                 object
Price                float64
dtype: object


In [282]:
#상관계수 분석
col = data.columns
col2 = []
for col1 in col:
    corr,p= st.pearsonr(data['Price'],data[col1])
    # p<0.05이고, corr가 0.5초과, -0.5미만인 특징 추출
    if (p<0.05) and ((corr>0.5) or (corr<-0.5)):
        print("Price와 {} 사이의 상관계수는 {:2f}이고, p-value는 {:2f}입니다.".format(col1, corr,p))
        col2.append(col1)
print(col2)

data=data.loc[:,col2]
data.head()

Price와 Engine 사이의 상관계수는 0.651252이고, p-value는 0.000000입니다.
Price와 Power 사이의 상관계수는 0.764591이고, p-value는 0.000000입니다.
Price와 Price 사이의 상관계수는 1.000000이고, p-value는 0.000000입니다.
['Engine', 'Power', 'Price']


,Engine,Power,Price
0,1582,126.20,12.50
1,1199,88.70,4.50
2,1248,88.76,6.00
3,1968,140.80,17.74
4,1461,63.10,3.50


In [236]:
#train, test 셋으로 나누기
train, test = train_test_split(data, train_size = 0.8)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
print(len(train))
print(len(test))

col = list(train.columns)
col.remove('Price')

trainY = train['Price'].to_numpy()
trainX = train.loc[:,col].to_numpy()

testY = test['Price'].to_numpy()
testX = test.loc[:,col].to_numpy()
trainX

4446
1112


array([[ 624. ,   35. ],
       [1998. ,  132. ],
       [1199. ,   73.9],
       ...,
       [2143. ,  170. ],
       [1995. ,  184. ],
       [1197. ,   85.8]])

In [283]:
import sklearn.preprocessing 
scaler = sklearn.preprocessing.MinMaxScaler()
trainX = scaler.fit_transform(trainX)
testX = scaler.fit_transform(testX)
print(trainX)

model = LinearRegression(fit_intercept = False, normalize=True, copy_X=True)
print(model)
model.fit(trainX, trainY)

print('R-square: {}'.format(model.score(trainX, trainY)))
print('R-square: {}'.format(model.score(testX, testY)))

[[0.         0.001545  ]
 [0.25567547 0.18887601]
 [0.10699665 0.07667053]
 ...
 [0.28265724 0.26226342]
 [0.25511723 0.28930089]
 [0.10662449 0.09965238]]
LinearRegression(fit_intercept=False, normalize=True)
R-square: 0.5590775298023285
R-square: 0.5242302225255642


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  FutureWarning,


In [247]:
#test데이터 불러오기
data_path = "/content/drive/My Drive/산업데이터과학/data/test_car.csv" 
datatest = pd.read_csv(data_path)
datatest.head()
datatest.columns
data_not_nullcolumns=['Engine', 'Power']
datatest=datatest.loc[:,data_not_nullcolumns]
print(datatest.dtypes)
datatest.head()

resX = datatest.loc[:].to_numpy()

resX = scaler.fit_transform(resX)

Engine      int64
Power     float64
dtype: object


In [248]:
testcar_predict = model.predict(resX)
res = pd.concat([pd.DataFrame(abs(testcar_predict)), pd.DataFrame(datatest)],axis=1)
res.columns=['Predict Price', 'Engine', 'Power']
print(res)

     Predict Price  Engine   Power
0        18.395140    1968  174.33
1         6.152890    1197   83.10
2        20.893082    2199  194.30
3        16.618497    1998  163.70
4        22.597939    2143  204.00
..             ...     ...     ...
245       6.113168    1197   82.85
246      18.395140    1968  174.33
247       8.222369    1598  103.50
248       4.182686     998   67.04
249       5.716195    1341   83.00

[250 rows x 3 columns]


In [249]:
res.to_csv('/content/drive/My Drive/산업데이터과학/data/test_car Predict.csv')